# Verify GPU

In [1]:
!nvidia-smi

Sat Apr 24 18:38:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Weight and Bias (Assisting Metrics, Optional)

In [2]:
!pip install wandb
!wandb login
project_name = "CoLA with BERT"  # @param {type:"string"}
import os

os.environ["WANDB_PROJECT"] = project_name
import wandb

     |████████████████████████████████| 2.1MB 8.5MB/s 
     |████████████████████████████████| 133kB 46.9MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 163kB 37.1MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=6223e85238566bbde576d8fa8686eed341c5d264925536a230e5d1e0a65beb19
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=ba23f4d52ce94865c1aaca33f149c929a1de21f7d14201534a47976515c49c6b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: A

# Installation

In [3]:
!pip install transformers==4.5.1 datasets==1.5.0 accelerate==0.2.1

     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 194kB 34.9MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 
     |████████████████████████████████| 901kB 47.1MB/s 
     |████████████████████████████████| 112kB 52.2MB/s 
     |████████████████████████████████| 245kB 52.3MB/s 


In [4]:
import os

import torch
from accelerate import Accelerator
from datasets import load_dataset, load_metric
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AdamW,
    BertConfig,
    BertModel,
    BertTokenizerFast,
    default_data_collator,
    get_linear_schedule_with_warmup,
)

In [5]:
per_device_train_batch_size = 32  # @param {type:"slider", min:1, max:64, step:1}
learning_rate = 2e-5  # @param {type:"slider", min:2e-7, max:2e-3, step:2e-7}
num_train_epochs = 3  # @param {type:"slider", min:1, max:10, step:1}
logging_steps = 10  # @param {type:"slider", min:10, max:100, step:5}

# Tokenizer

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

# Dataset

In [7]:
datasets = load_dataset("glue", "cola")
label_list = datasets["train"].features["label"].names

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.


In [8]:
def preprocess_function(examples):
    # Tokenize the texts
    result = tokenizer(
        examples["sentence"],
        padding="max_length",
        max_length=128,
        truncation="longest_first",
    )
    result["label"] = examples["label"]
    return result

In [9]:
datasets = datasets.map(preprocess_function, batched=True)
datasets["test"] = datasets["test"].remove_columns("label")

In [10]:
for index in range(3):
    print(f"Sample {index} of the training set: {datasets['train'][index]}.")

Sample 0 of the training set: {'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'idx': 0, 'input_ids': [101, 3458, 2053, 1281, 112, 189, 4417, 1142, 3622, 117, 1519, 2041, 1103, 1397, 1141, 1195, 17794, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'toke

In [11]:
dataloaders = {
    "train": DataLoader(
        datasets["train"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
    "validation": DataLoader(
        datasets["validation"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
    "test": DataLoader(
        datasets["test"],
        collate_fn=default_data_collator,
        batch_size=per_device_train_batch_size,
        shuffle=True,
    ),
}

# Model

In [3]:
class BertClassificationForCoLA(nn.Module):
    def __init__(self, config, from_pretrained=False):
        super().__init__()
        self.config = config
        self.num_labels = config.num_labels

        if not from_pretrained:
            self.bert = BertModel(config)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.apply(self._init_weights)  # Equivelent of self.init_weights()

    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    @classmethod
    def from_pretrained(cls, model_name: str, config):
        model = cls(config, from_pretrained=True)
        model.bert = BertModel.from_pretrained(model_name, config=config)
        return model

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = self.dropout(outputs[1])
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # return final
        return {
            "loss": loss,
            "logits": logits,
        }

In [13]:
config = BertConfig.from_pretrained(
    "bert-base-cased", num_labels=len(label_list), finetuning_task="cola"
)
model = BertClassificationForCoLA.from_pretrained("bert-base-cased", config)

# Optimizer, scheduler, accelerator

In [14]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [
            p
            for n, p in model.named_parameters()
            if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.01,
    },
    {
        "params": [
            p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
num_warmup_steps = 0
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_train_epochs * len(dataloaders["train"])
)

In [15]:
accelerator = Accelerator()
(
    model,
    optimizer,
    dataloaders["train"],
    dataloaders["validation"],
    dataloaders["test"],
) = accelerator.prepare(
    model,
    optimizer,
    dataloaders["train"],
    dataloaders["validation"],
    dataloaders["test"],
)

# Training

In [16]:
run_name = "PyTorch Scratch"
if os.getenv("WANDB_PROJECT"):
    wandb.init(project=project_name, name=run_name)
    wandb.watch(model)
for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(tqdm(dataloaders["train"])):
        batch.pop("idx")

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs["loss"]
        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()

        if step % logging_steps == 0:
            wandb.log(
                {
                    "train/loss": loss,
                    "train/epoch": epoch + step / len(dataloaders["train"]),
                    "train/learning_rate": scheduler.get_last_lr()[0],
                }
            )
    model.eval()
    metric = load_metric("glue", "cola")
    for batch in tqdm(dataloaders["validation"]):
        batch.pop("idx")

        outputs = model(**batch)
        outputs = accelerator.gather(outputs)
        predictions = outputs["logits"].argmax(dim=-1)
        references = accelerator.gather(batch["labels"])

        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    wandb.log({"eval/matthews_correlation": eval_metric["matthews_correlation"]})

os.mkdir(run_name)
accelerator.wait_for_everyone()
model = accelerator.unwrap_model(model)
accelerator.save(model.state_dict(), os.path.join(run_name, "trained_model.pt"))

wandb: Currently logged in as: tomy0000000 (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


# Test

## On dataset

In [17]:
predictions = torch.empty(0, dtype=torch.int64, device=accelerator.device)
for batch in tqdm(dataloaders["test"]):
    batch.pop("idx")
    outputs = model(**batch)
    predictions = torch.cat((predictions, outputs["logits"].argmax(dim=-1)), 0)

In [18]:
for index, (sample, pred) in enumerate(zip(datasets["test"]["sentence"], predictions)):
    print(f"{index}\t{label_list[pred.item()]}\t{sample}")

0	acceptable	Bill whistled past the house.
1	acceptable	The car honked its way down the road.
2	acceptable	Bill pushed Harry off the sofa.
3	acceptable	the kittens yawned awake and played.
4	acceptable	I demand that the more John eats, the more he pay.
5	acceptable	If John eats more, keep your mouth shut tighter, OK?
6	acceptable	His expectations are always lower than mine are.
7	acceptable	The sooner you call, the more carefully I will word the letter.
8	unacceptable	The more timid he feels, the more people he interviews without asking questions of.
9	unacceptable	Once Janet left, Fred became a lot crazier.
10	acceptable	If you give every senator enough opportunity, he will succumb to corruption.
11	acceptable	The more time that any senator spends with lobbyists, the more likely he is to succumb to corruption.
12	acceptable	Which problem does the sooner you solve, the more easily you'll satisfy the folks up at corporate headquarters?
13	acceptable	Which problem do you think that the s

## Manually

In [19]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(accelerator.device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs['logits'].argmax(dim=-1).item()]}")

Prediction: acceptable


# Inference

In [1]:
import torch
from torch import nn
from transformers import BertConfig, BertModel, BertTokenizerFast

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label_list = ["unacceptable", "acceptable"]
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
config = BertConfig.from_pretrained("bert-base-cased", finetuning_task="cola")

In [4]:
# Scroll to top and run the class definition class first
model = BertClassificationForCoLA.from_pretrained(
    "PyTorch Scratch/trained_model.pt", config
).to(device)

In [5]:
sentence = "The probable hostile German reaction is unfortunate."  # @param {type:"string"}
tokenized_input = tokenizer(sentence, return_tensors="pt").to(device)
outputs = model(**tokenized_input)
print(f"Prediction: {label_list[outputs['logits'].argmax(dim=-1).item()]}")

Prediction: unacceptable
